In [ ]:
fefe 

<div style="background-color:blue; opacity:0.5; padding:6px">
<div style="background-color:blue; opacity:0.5; padding:6px; color:white; font-weight:bold">
<h1><em>Introduction</em></h1>
</div>


In [ ]:
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
import numpy as np

# Step 2: Text Extraction
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        num_pages = pdf_reader.numPages
        for page_num in range(num_pages):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
    return text

# Step 3: Data Preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Step 4: Feature Engineering (TF-IDF)
def tfidf_vectorize(text_data):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)
    return tfidf_matrix

# Step 5: Model Training (Logistic Regression)
def train_logistic_regression_model(X_train, y_train):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    return model

# Step 6: Model Training (Random Forest)
def train_random_forest_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

# Step 7: Model Training (SVM)
def train_svm_model(X_train, y_train):
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    return model

# Step 8: Model Training (Multinomial Naive Bayes)
def train_multinomial_nb_model(X_train, y_train):
    model = MultinomialNB()
    model.fit(X_train, y_train)
    return model

# Step 9: Train Word2Vec model
def train_word2vec_model(sentences):
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    return word2vec_model

# Step 10: Transform text data into word embeddings
def text_to_embeddings(text, word2vec_model):
    tokens = word_tokenize(text)
    embeddings = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)  # Return zero vector if no embeddings found

# Step 11: Combine predictions using majority voting
class BlendedModel(BaseEstimator, ClassifierMixin):
    def _init_(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        blended_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
        return blended_pred